In [18]:
import json
import textwrap
import os

def lower_no_spaces(s):
    return s.lower().replace(' ','-')

bios_string = textwrap.dedent('''\
+++
date = '2025-01-24T21:30:10-05:00'
draft = false
title = 'Bios'
+++

''')

with open('bios.json') as f:
    bios_data = json.load(f)
    for name,bio in bios_data.items():
        bios_string += f'## {name}\n\n{bio}\n\n'
        
with open('bios.md', 'w') as f:
    f.write(bios_string)
    
os.system('cp bios.md ../content/bios.md')
    
def title_location_time(obj,day_string):
    return textwrap.dedent(f'''
    ---
    ## {obj['title']}  
    {obj['time']}  
    {obj['location']}
    ''')
    
def keynote(obj,day_string):
    return textwrap.dedent(f'''
    ---
    ## {obj['title']}  
    {obj['time']}  
    {obj['location']}
    
    {bios_data[obj['speaker']]}
    ''')
    
def work(obj):
    
    work_string = '\n\n'
    if 'description' in obj:
        work_string += f'{obj['description']}  \n'
    else:
        work_string += f'***{obj['title']}*** by {obj['composer']}  \n'
    
    for performer in obj['performers']:    
        work_string += f'&emsp;&emsp;*{performer}*  \n'
        
    work_string += '\n'
    return work_string

def write_to_content(string,file_name):
    with open(f'{file_name}', 'w') as f:
        f.write(f'/content/{string}')

performer_avoiders = {'???', 'fixed media', 'Workshop Participants'}

def add_person(set,perf):
    if perf not in performer_avoiders:
        person = perf.split(',')[0]
        set[person] = None

def concert_page(obj,day_string):
    
    performers = {}
    composers = {}
        
    concert_page = textwrap.dedent(f'''
    +++
    date = '2025-01-24T21:30:10-05:00'
    draft = false
    title = '{obj['title']}'
    +++
    
    {day_string}  
    {obj['time']}  
    {obj['location']}
    ''')
    
    for w in obj['works']:
        for p in w['performers']:
            add_person(performers,p)
        concert_page += work(w)
        
    concert_page += f'---\n\n## Program Notes\n\n'
    
    for w in obj['works']:
        
        concert_page += f'### *{w["title"]}*\n\n'
        
        if 'bios' in w:
            concert_page += 'by:  \n'
            for b in w['bios']:
                concert_page += f'[{b}](/bios/#{lower_no_spaces(b)})  \n'
                add_person(composers,b)
            concert_page += '\n\n'
        else:
            add_person(composers,w['composer'])
            concert_page += f'by [{w["composer"]}](/bios/#{lower_no_spaces(w["composer"])})\n\n'
        
        concert_page += w["program-note"] + '\n\n'
        
    for key in list(composers.keys()):
        performers.pop(key, None) 
    
    if len(performers) > 0:
        concert_page += f'---\n\n## Performer Bios\n\n'
        
        for p in performers.keys():
            concert_page += f'[{p}](/bios/#{lower_no_spaces(p)})  \n'
        
        concert_page += '\n'
    
    write_to_content(concert_page,lower_no_spaces(obj['title']) + '.md')

def concert(obj,day_string):
    
    concert_page(obj,day_string)
    
    concert_string = textwrap.dedent(f'''
    ---
    ## {obj['title']}  

    {obj['time']}  
    {obj['location']}  
    [go to program](/{lower_no_spaces(obj['title'])})
    
    ''')
    
    for w in obj['works']:
        concert_string += work(w)
    
    return concert_string

def paper(obj):
    return f'**{obj["title"]}**  \n&emsp;&emsp;*{obj['author']}*  \n&emsp;&emsp;{obj["time"]}\n\n'
    
def paper_session_page(obj,day_string):
    string = textwrap.dedent(f'''
    +++
    date = '2025-01-24T21:30:10-05:00'
    draft = false
    title = '{obj['title']}'
    +++
    
    {day_string}  
    {obj['time']}  
    {obj['location']}
    
    ''')
    
    for p in obj['presenters']:
        string += f'## {p['title']}\n\n'
        string += f'[{p["author"]}](/bios/#{lower_no_spaces(p['author'])})  \n'
        string += f'{p['time']}\n\n'
        string += p['abstract'] + '\n\n'
    
    write_to_content(string,lower_no_spaces(obj['title']) + '.md')
        

def paper_session(obj,day_string):
    
    paper_session_page(obj,day_string)
    
    paper_string = textwrap.dedent(f'''
    ---
    ## {obj['title']}  
    {obj['time']}  
    {obj['location']}  
    [go to abstracts](/{lower_no_spaces(obj['title'])}/)
    
    ''')
    
    for p in obj['presenters']:
        paper_string += paper(p)
        
    return paper_string
    
def workshop(obj,day_string):
    workshop_string = textwrap.dedent(f'''
    ---
    ## {obj['title']}  
    {obj['time']}
    
    *Both workshops are one hour and occur simultaneously.*
    
    ''')
    
    for workshop in obj['workshops']:
        workshop_string += textwrap.dedent(f'''
        ### {workshop['title']}
        
        *Led by {workshop['facilitator']}*  
        {workshop['location']}  
        
        ''')
    
    return workshop_string
    
def installations(obj,day_string):
    installations_string = textwrap.dedent(f'''
    ---
    ## {obj['title']}  
    {obj['time']}  
    
    ''')
    
    for installation in obj['installations']:
        installations_string += textwrap.dedent(f'''
        &emsp;&emsp;**{installation['title']}**  
        &emsp;&emsp;&emsp;&emsp;*by {installation['artist']}*  
        &emsp;&emsp;&emsp;&emsp;{installation['location']}
        ''')
        
    return installations_string
    
event_processors = {
    'coffee': title_location_time,
    'keynote': keynote,
    'paper': paper_session,
    'workshop': workshop,
    'concert': concert,
    'installations': installations,
    'other': title_location_time
}

full_schedule = textwrap.dedent(f'''
        +++
        date = '2025-01-24T21:30:10-05:00'
        draft = false
        title = 'Full Schedule'
        +++
        
        ''')

with open('program-data.json') as f:
    data = json.load(f)
    for i,day in enumerate(data['days']):
        day_prefix = textwrap.dedent(f'''
        +++
        date = '2025-01-24T21:30:10-05:00'
        draft = false
        title = 'Day {i+1}'
        +++
        
        ''')
        
        day_string = f'**{day['day']}**\n\n'
        
        for event in day['events']:
            day_string += event_processors[event['type']](event,day['day'])
            
        full_schedule += f'\n\n---\n\n# Day {i+1}\n\n' + day_string
            
        write_to_content(day_prefix + day_string,f'day-{i+1}.md')

write_to_content(full_schedule,'full-schedule.md')

        